In [2]:
import pandas as pd
import os

# Loading the data

In [3]:
from se_kge.default_predictor import predictor, embeddings_path, graph_path, model_path, mapping_path

In [4]:
print(f"""Loaded default predictor using paths:

embeddings: {embeddings_path}
graph: {graph_path}
model: {model_path}
mapping: {mapping_path}

""")

Loaded default predictor using paths:

embeddings: C:\Users\Rana Aldisi\Documents\GitHub\SE_KGE\resources\predictive_model\070819_node2vec_embeddings_complete01.embeddings
graph: C:\Users\Rana Aldisi\Documents\GitHub\SE_KGE\resources\chemsim_50_graphs\fullgraph_with_chemsim_50.edgelist
model: C:\Users\Rana Aldisi\Documents\GitHub\SE_KGE\resources\predictive_model\070819_node2vec_model_complete01.pkl
mapping: C:\Users\Rana Aldisi\Documents\GitHub\SE_KGE\resources\mapping\fullgraph_nodes_mapping.tsv




# Examples of different kinds of predictions with literature evidence

## side effect - target association

In [5]:
predictor.find_new_relation(
    entity_name_1='EGFR',
    entity_name_2='Papulopustular rash',
)
#PMID: 18165622

{'entity_1': {'node_id': 8499,
  'namespace': 'uniprot',
  'name': 'EGFR',
  'identifier': 'P00533'},
 'entity_2': {'node_id': 6791,
  'namespace': 'umls',
  'name': 'Papulopustular rash',
  'identifier': 'C2609319'},
 'probability': 0.02084655412288939}

In [6]:
predictor.find_new_relation(
node_id_1=9438, #Histamine receptor H1
node_id_2=331, #Drowsiness
)
#PMID: 26626077

{'entity_1': {'node_id': 9438,
  'namespace': 'uniprot',
  'name': 'HRH1',
  'identifier': 'P35367'},
 'entity_2': {'node_id': 331,
  'namespace': 'umls',
  'name': 'Drowsiness',
  'identifier': 'C0013144'},
 'probability': 0.022773732615890885}

## drug- side effect association

In [7]:
predictor.find_new_relation(
node_id_1='3534', #diazepam
node_id_2='670', #Libido decreased
)
#PMID: 29888057

{'entity_1': {'node_id': '3534',
  'namespace': 'pubchem',
  'name': '7-chloro-1-methyl-5-phenyl-3H-1,4-benzodiazepin-2-one',
  'identifier': '3016'},
 'entity_2': {'node_id': '670',
  'namespace': 'umls',
  'name': 'Libido decreased',
  'identifier': 'C0011124'},
 'probability': 0.9969995536047346}

In [9]:
predictor.find_new_relation(
node_id_1=10270, #Cytarabine 
node_id_2=1149, #Anaemia megaloblastic
)
# PMID: 23157436

{'entity_1': {'node_id': 10270,
  'namespace': 'pubchem',
  'name': '4-amino-1-[(2R,3S,4S,5R)-3,4-dihydroxy-5-(hydroxymethyl)oxolan-2-yl]pyrimidin-2-one',
  'identifier': '6253'},
 'entity_2': {'node_id': 1149,
  'namespace': 'umls',
  'name': 'Anaemia megaloblastic',
  'identifier': 'C0002888'},
 'probability': 0.0007503847088747362}

## drug-target association

In [10]:
predictor.find_new_relation(
node_id_1=12765, #Sertindole 
node_id_2=9372,  #CHRM1 receptor
)
# PMID: 29942259 

{'entity_1': {'node_id': 12765,
  'namespace': 'pubchem',
  'name': '1-[2-[4-[5-chloro-1-(4-fluorophenyl)indol-3-yl]piperidin-1-yl]ethyl]imidazolidin-2-one',
  'identifier': '60149'},
 'entity_2': {'node_id': 9372,
  'namespace': 'uniprot',
  'name': 'CHRM1',
  'identifier': 'P11229'},
 'probability': 0.014544440338331974}

# Example of predicting relations using node2vec model and embeddings

In [ ]:
results = predictor.find_new_relations(
    entity_identifier='145742',
    entity_type='phenotype',
    k=10,
)
print(results['query'])
results_df = pd.DataFrame(results['predictions'])
results_df

creating relations list:  27%|███████████████████████████████▍                                                                                   | 3755/13730 [00:18<02:38, 62.95it/s]